In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from helpers import (get_training_observations, 
                     get_training_labels)
import pandas as pd
import numpy as np
from joblib import dump
from joblib import load

# Model specific imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import metrics
tf.get_logger().setLevel('INFO')

import optuna

In [4]:
# Set up data
x_train = get_training_observations()
y_train = get_training_labels()
le = load('../model_joblibs/final_labelEncoder.joblib')
    
y_train_transformed = le.transform(y_train['EMPO_3'])
y_train_transformed

x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train_transformed, test_size=0.2, random_state=0, stratify=y_train_transformed)   


Getting all training observations from 'metagenome_classification.db'...
Getting all training labels from 'metagenome_classification.db'...


In [5]:
K = tf.keras.backend

def build_model(
    activation='relu',
    hidden_layer_sizes=[],
    final_layer_activation='softmax',
    dropout=0.0,
    optimizer='Adam',
    learning_rate=0.01,
    kernel_regularizer=1e-5,
    bias_regularizer=1e-5,
    activity_regularizer=1e-5,
    n_classes=15
):
    """Build a multi-class logistic regression model using Keras.

    Args:
      n_classes: Number of output classes in the dataset.
      hidden_layer_sizes: A list with the number of units in each hidden layer.
      activation: The activation function to use for the hidden layers.
      optimizer: The optimizer to use (SGD, Adam).
      learning_rate: The desired learning rate for the optimizer.

    Returns:
      model: A tf.keras model (graph).
    """
    tf.keras.backend.clear_session()
    np.random.seed(0)
    tf.random.set_seed(0)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    
    for hidden_layer_size in hidden_layer_sizes:
        if hidden_layer_size > n_classes:
            model.add(tf.keras.layers.Dense(
                hidden_layer_size,
                activation=activation,
                kernel_regularizer=tf.keras.regularizers.L2(kernel_regularizer),
                bias_regularizer=tf.keras.regularizers.L2(bias_regularizer),
                activity_regularizer=tf.keras.regularizers.L2(activity_regularizer)
            ))
            if dropout > 0:
                model.add(tf.keras.layers.Dropout(dropout))

    model.add(tf.keras.layers.Dense(
        n_classes,
        activation=final_layer_activation,
        kernel_regularizer=tf.keras.regularizers.L2(kernel_regularizer),
        bias_regularizer=tf.keras.regularizers.L2(bias_regularizer),
        activity_regularizer=tf.keras.regularizers.L2(activity_regularizer)
    ))
    opt = None
    if optimizer == "SGD":
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "Adam":
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        raise f"Unsupported optimizer, {optimizer}"
    model.compile(
        loss="categorical_crossentropy", optimizer=opt, 
        metrics=[tfa.metrics.F1Score(num_classes=15, average='weighted')]
    )
    return model
    

In [6]:
def objective_1_layer(trial, x_tr=x_tr, x_val=x_val, y_tr=y_tr, y_val=y_val):
    
    # Set up XGBoost hyperparameter space
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.01),
        'hidden_layer_size1': trial.suggest_int('hidden_layer_size1', 0, 1024),
        'kernel_regularizer': trial.suggest_float('kernel_regularizer', 1e-10, 1e-4),
        'bias_regularizer': trial.suggest_float('bias_regularizer', 1e-10, 1e-4),
        'activity_regularizer': trial.suggest_float('activity_regularizer', 1e-10, 1e-4),
        'dropout': trial.suggest_float('dropout', 0, 0.1)
    }
    
    hidden_layer_sizes = [params.pop('hidden_layer_size1')]
    
    y_tr_one_hot = K.one_hot(K.cast(y_tr, dtype='int64'), 15)
    y_val_one_hot = K.one_hot(K.cast(y_val, dtype='int64'), 15)
    
    # Create model
    nn = build_model(
        **params,
        n_classes=15,
        optimizer='Adam',
        final_layer_activation='softmax',
        hidden_layer_sizes=hidden_layer_sizes
    )
    
    # Fit model
    nn.fit(x=x_tr,
           y=y_tr_one_hot,
           epochs=50,
           batch_size=64,
           verbose=0)
    
    evaluation = nn.evaluate(x=x_val, y=y_val_one_hot, verbose=0, return_dict=True)
    f1_score = evaluation['f1_score']
    loss = evaluation['loss']
    
    print(f'F1 Score: {f1_score}; Loss={loss}')
    
    return f1_score
    

In [7]:
study_nn_1 = optuna.create_study(direction='maximize')
study_nn_1.optimize(objective_1_layer, n_trials=100)

print('Number of finished trials:', len(study_nn_1.trials))
print('Best trial:', study_nn_1.best_trial.params)

[I 2022-08-03 12:27:33,886] A new study created in memory with name: no-name-e01a18be-2164-4735-a53a-cca45992ccdd
2022-08-03 12:27:33.895479: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-03 12:27:33.895877: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-08-03 12:27:34.003207: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Metal device set to: Apple M1 Pro


2022-08-03 12:27:34.458202: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-03 12:27:45.545452: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
[I 2022-08-03 12:27:45,716] Trial 0 finished with value: 0.7691457867622375 and parameters: {'learning_rate': 0.00953325013159356, 'hidden_layer_size1': 766, 'kernel_regularizer': 6.88890159678785e-05, 'bias_regularizer': 4.725846676750513e-05, 'activity_regularizer': 5.822785882696898e-05, 'dropout': 0.03081991436366728}. Best is trial 0 with value: 0.7691457867622375.


F1 Score: 0.7691457867622375; Loss=1.0762218236923218


2022-08-03 12:27:46.040208: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-03 12:27:53.543880: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
[I 2022-08-03 12:27:53,699] Trial 1 finished with value: 0.7330021858215332 and parameters: {'learning_rate': 0.00567869353715606, 'hidden_layer_size1': 136, 'kernel_regularizer': 3.3137920656898724e-05, 'bias_regularizer': 3.8474014255903916e-05, 'activity_regularizer': 2.8087369320368406e-05, 'dropout': 0.09083281167360414}. Best is trial 0 with value: 0.7691457867622375.


...

F1 Score: 0.8731788396835327; Loss=0.5912575721740723


2022-08-03 13:02:14.107729: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-03 13:02:37.572336: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
[I 2022-08-03 13:02:38,311] Trial 98 finished with value: 0.9070559740066528 and parameters: {'learning_rate': 0.005911042371411944, 'hidden_layer_size1': 553, 'kernel_regularizer': 1.3369437767420948e-08, 'bias_regularizer': 9.80464976520079e-05, 'activity_regularizer': 4.589242948117302e-05, 'dropout': 0.07062744423822057}. Best is trial 48 with value: 0.9458847641944885.


F1 Score: 0.9070559740066528; Loss=0.3481776714324951


2022-08-03 13:02:38.807013: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-03 13:03:14.156818: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
[I 2022-08-03 13:03:15,000] Trial 99 finished with value: 0.84844970703125 and parameters: {'learning_rate': 0.009371599620245185, 'hidden_layer_size1': 436, 'kernel_regularizer': 1.2094243508471935e-05, 'bias_regularizer': 9.871898403147731e-05, 'activity_regularizer': 3.432117518507776e-05, 'dropout': 0.07707651873284366}. Best is trial 48 with value: 0.9458847641944885.


F1 Score: 0.84844970703125; Loss=0.7259964346885681
Number of finished trials: 100
Best trial: {'learning_rate': 0.009590263929390183, 'hidden_layer_size1': 663, 'kernel_regularizer': 3.224294748659483e-08, 'bias_regularizer': 9.494455246549682e-05, 'activity_regularizer': 7.54622803748681e-05, 'dropout': 0.07189361802417232}


Best trial params:
* {'learning_rate': 0.008842046029948113, 'hidden_layer_size1': 735, 'kernel_regularizer': 3.25276114331844e-08, 'bias_regularizer': 5.484746504195214e-05, 'activity_regularizer': 9.161273662322264e-05, 'dropout': 0.03432482015904013}

* **With stratified split:** Trial 48 finished with value: 0.9458847641944885 and parameters: {'learning_rate': 0.009590263929390183, 'hidden_layer_size1': 663, 'kernel_regularizer': 3.224294748659483e-08, 'bias_regularizer': 9.494455246549682e-05, 'activity_regularizer': 7.54622803748681e-05, 'dropout': 0.07189361802417232}. Best is trial 48 with value: 0.9458847641944885.